## Interact with BiG-SLICE SQL Query result

In [1]:
import sqlite3
import pandas as pd
from os import path
import json

## Download bigslice query from azure

In [2]:
# load credentials
with open("../secret/blob-key.json") as f:
    az_account = json.load(f)
account_name = az_account["account_name"]
key = az_account["key"]

In [3]:
# fetch from azure
container = "processed"
item_to_download = "bigslice/bigslice_query.zip"
target = "../data/bigslice_query.zip"
! az Storage blob download --account-key {key} --account-name {account_name} \
    --container-name {container} --file {target} --name {item_to_download} \
    > ../data/bigslice_query.log

Finished[#############################################################]  100.0000%


In [13]:
# extract data
! (cd ../data/ && unzip -f bigslice_query.zip)

Archive:  bigslice_query.zip


## Interact with sql db via sqlite3

In [5]:
# initiate connection
result_folder = "../data/full_run_result/"
conn = sqlite3.connect(path.join(result_folder, "reports/1/data.db"))

In [6]:
# get all table name from sql
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
sql_table_names = [i[0] for i in cursor.fetchall()]
sql_table_names

['schema',
 'bgc',
 'sqlite_sequence',
 'cds',
 'bgc_class',
 'hsp',
 'hsp_alignment',
 'hsp_subpfam',
 'bgc_features',
 'gcf_membership']

In [7]:
# convert table to pandas df
df_tables = []
for t in sql_table_names:
    df = pd.read_sql_query(f"select * from {t};", conn)
    df_tables.append(df)

### EDA

In [39]:
# slice 1 example BGCs
df_input = df_tables[1]
df_input[df_input.id == 1]

,id,name,type,on_contig_edge,length_nt,orig_folder,orig_filename
0,1,data/interim/bigslice/tmp/f__Nitrospiraceae_al...,as6,1,20556,data/interim/bigslice/tmp/f__Nitrospiraceae_al...,VOPV01000005.1.region001.gbk


In [40]:
# see 10 top hits to BGCs
df_gcf = df_tables[9]
df_gcf[df_gcf.bgc_id == 1]

,gcf_id,bgc_id,membership_value,rank
5470,244878,1,1176,0
5471,245671,1,1282,1
5472,247890,1,1328,2
5473,245936,1,1346,3
5474,246216,1,1353,4
5475,245560,1,1362,5
5476,246221,1,1370,6
5477,246515,1,1437,7
5478,245666,1,1449,8
5479,246695,1,1526,9
